In [1]:
import pandas as pd 
import numpy as np 
import os 
import xlrd 
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate 

In [ ]:
os.environ["OPENAI_API_KEY"] = "API Key"
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0.3)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_8672\14006444.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0.3)


In [3]:
df = pd.read_excel("C:\\Users\\LENOVO\\Downloads\\Chinese_addresses.xlsx")

In [4]:
df.head()

,Company Name,Street Name,Place,City,Pincode
0,Wipro Chengdu Limited,天府大道南段599号天府软件园D2栋,天府软件园,成都,610041
1,Wipro Dalian Limited,甘井子区亿达春田BEST城D7栋,亿达春田BEST城,大连,116033
2,Cognizant Technology Solutions,汇贤园9号楼6-9层，腾飞软件园,高新技术产业园区,大连,-


In [5]:
df.shape

(3, 5)

In [6]:
print(df.columns)


Index(['Company Name', 'Street Name', 'Place', 'City', 'Pincode'], dtype='object')


In [22]:
from langchain.schema import SystemMessage, HumanMessage

def get_area(street, city):
    """"Use OPENAI to predict the missing area."""
    address = f"{street},{city}"
    messages = [
        SystemMessage(content="You are an AI assistant that extracts area information form English addresses."),
        HumanMessage(content=f"What is the area in the following address? {address}\n return only with area name.")
    ]
    response = llm(messages)
    return response.content.strip() if response.content else ""

In [8]:
df["Place"] = df.apply(lambda row: get_area(row["City"], row["Chengdu"]) if pd.isna(row["Place"]) else row ["Place"], axis = 1)

In [9]:
df.head()

,Company Name,Street Name,Place,City,Pincode
0,Wipro Chengdu Limited,天府大道南段599号天府软件园D2栋,天府软件园,成都,610041
1,Wipro Dalian Limited,甘井子区亿达春田BEST城D7栋,亿达春田BEST城,大连,116033
2,Cognizant Technology Solutions,汇贤园9号楼6-9层，腾飞软件园,高新技术产业园区,大连,-


In [10]:
df["Place"].head(15)

0        天府软件园
1    亿达春田BEST城
2     高新技术产业园区
Name: Place, dtype: object

In [11]:
if "raajy" not in df.columns:
    df["raajy"] = None

In [12]:
def get_state(pincode, street, city):
    """Use OpenAI to predict the state based on pincode, street, city"""
    address1 = f"{pincode},{street},{city}"
    messages = [
        SystemMessage(content="You are an AI assistant that extracts state information from English addresses."),
        HumanMessage(content=f"What is the state in the following address? {address1}\n return only with state name in English.")
    ]
    response = llm(messages)
    return response.content.strip() if response.content else ""

In [13]:
df["raajy"] = df.apply(lambda row: get_state(row["Pincode"], row["Place"], row["City"]) if pd.isna(row["raajy"]) else row ["raajy"], axis = 1)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_8672\3875321795.py:8: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)


In [14]:
df.head()

,Company Name,Street Name,Place,City,Pincode,raajy
0,Wipro Chengdu Limited,天府大道南段599号天府软件园D2栋,天府软件园,成都,610041,"The state in the address ""610041,天府软件园,成都"" is ..."
1,Wipro Dalian Limited,甘井子区亿达春田BEST城D7栋,亿达春田BEST城,大连,116033,The state in the address provided is Liaoning.
2,Cognizant Technology Solutions,汇贤园9号楼6-9层，腾飞软件园,高新技术产业园区,大连,-,The state in the address is Liaoning.


In [15]:
df['raajy'].head(15)

0    The state in the address "610041,天府软件园,成都" is ...
1       The state in the address provided is Liaoning.
2                The state in the address is Liaoning.
Name: raajy, dtype: object

In [16]:
if "Country" not in df.columns:
    df["Country"] = None

In [17]:
def get_country(state, city, pincode):
    """Use OpenAI to predict the country based on state, city, pincode."""
    address2 = f"{state},{city}, {pincode}"
    messages = [
        SystemMessage(content="You are an AI assistant that extracts country information from English addresses."),
        HumanMessage(content=f"What is the country in the following address? {address2}\n return only with country name in English.")
    ]

    response = llm(messages)
    return response.content.strip() if response.content else ""

In [18]:
df["Country"] = df.apply (lambda row: get_country(row["Pincode"], row["raajy"], row["City"]) if pd.isna(row["Country"]) else row ["Country"], axis = 1)

In [19]:
df.head()

,Company Name,Street Name,Place,City,Pincode,raajy,Country
0,Wipro Chengdu Limited,天府大道南段599号天府软件园D2栋,天府软件园,成都,610041,"The state in the address ""610041,天府软件园,成都"" is ...","The country in the address ""610041, 天府软件园, 成都""..."
1,Wipro Dalian Limited,甘井子区亿达春田BEST城D7栋,亿达春田BEST城,大连,116033,The state in the address provided is Liaoning.,The country in the provided address is China.
2,Cognizant Technology Solutions,汇贤园9号楼6-9层，腾飞软件园,高新技术产业园区,大连,-,The state in the address is Liaoning.,The country in the address you provided is China.


In [20]:
df['Country'].head(15)

0    The country in the address "610041, 天府软件园, 成都"...
1        The country in the provided address is China.
2    The country in the address you provided is China.
Name: Country, dtype: object

In [21]:
#df.to_excel("C:\\Users\\LENOVO\\Downloads\\updated_company_addresses_English.xlsx", index=False)
